In [1]:
import Learn2_new as ln
ut = ln.ut # utilities
ef = ln.ef # ERA_Fields_New

# log to stdout
import logging
import sys
logging.getLogger().level = logging.INFO
logging.getLogger().handlers = [logging.StreamHandler(sys.stdout)]

2022-02-17 09:01:42.577165: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
In this environment you cannot import Basemap
In this environment you cannot import Basemap


## Dealing with parameters

The structure of `Learn2_new.py` is pretty nested, with functions calling other functions in such a way that changing a parameter seems difficult.
The best way to do so is by using the functions `ln.get_default_params` and `ut.set_values_recursive`

### Beginner approach

The first way of approaching the code is by looking at the documentation of its functions, and when they have an argument of kind `*_kwargs`, it means that the function `*` will be called, so you can then look at its documentation and so on

Let's say you want to use the function `ln.prepare_data`. If you look at its documentation you get

In [2]:
help(ln.prepare_data)

Help on function prepare_data in module Learn2_new:

prepare_data(load_data_kwargs=None, prepare_XY_kwargs=None)
    Combines all the steps from loading the data to the creation of X and Y
    
    Parameters
    ----------
    load_data_kwargs: dict
        arguments to pass to the function `load_data`
    prepare_XY_kwargs: dict
        arguments to pass to the function `prepare_XY`
        
    Returns
    -------
    X : np.ndarray
        data. If flatten_time_axis with shape (days, lat, lon, fields), else (years, days, lat, lon, fields)
    Y : np.ndarray 
        labels. If flatten_time_axis with shape (days,), else (years, days)
    year_permutation : np.ndarray
        with shape (years,), final permutaion of the years that reproduces X and Y once applied to the just loaded data



This functions calls `ln.load_data` and `ln.prepare_XY`.

In [3]:
help(ln.load_data)

Help on function load_data in module Learn2_new:

load_data(dataset_years=8000, year_list=None, sampling='', Model='Plasim', area='France', filter_area='France', lon_start=0, lon_end=128, lat_start=0, lat_end=22, mylocal='/local/gmiloshe/PLASIM/', fields=['t2m', 'zg500', 'mrso_filtered'])
    Loads the data into Plasim_Fields objects
    
    Parameters
    ----------
    dataset_years : int, optional
        number of years of the dataset, for now 8000 or 1000.
    year_list : array-like or str or int or tuple or None, optional
        list of years to load from the dataset. If None all years are loaded
        if str must be in the format 'range([<start>],<end>,[<step>])', where square brackets mean the argument is optional. It will be interpreted as np.range([<start>],<end>,[<step>])
        if tuple must be in format ([<start>],<end>,[<step>])
        if int is just like providing only <end>
    sampling : str, optional
        '' (dayly) or '3hrs'
    Model : str, optional
       

Note that the variable called `mylocal` governs the source of data. Depending on the machine we are using we might want to change it. For instance on r740server5, the local space is acutally: '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/PLASIM/';

In [4]:
help(ln.prepare_XY)

Help on function prepare_XY in module Learn2_new:

prepare_XY(fields, make_XY_kwargs=None, roll_X_kwargs=None, do_premix=False, premix_seed=0, do_balance_folds=True, nfolds=10, year_permutation=None, flatten_time_axis=True)
    Performs all operations to extract from the fields X and Y ready to be fed to the neural network.
    
    Parameters
    ----------
    fields : dict of ef.Plasim_Field objects
    make_XY_kwargs : dict
        arguments to pass to the function `make_XY`
    roll_X_kwargs : dict
        arguments to pass to the function `roll_X`
    do_premix : bool, optional
        whether to perform premixing, by default False
    premix_seed : int, optional
        seed for premixing, by default 0
    do_balance_folds : bool, optional
        whether to balance folds
    nfolds : int, optional
        necessary for balancing folds
    year_permutation : np.ndarray, optional
        if provided overrides both premixing and fold balancing, useful for transfer learning as avoi

And `ln.prepare_XY` calls `ln.make_XY` and `ln.roll_X`

In [5]:
help(ln.make_XY)

Help on function make_XY in module Learn2_new:

make_XY(fields, label_field='t2m', time_start=30, time_end=120, T=14, tau=0, percent=5, threshold=None)
    Combines `make_X` and `assign_labels`
    
    Parameters:
    -----------
    fields : dict of Plasim_Field objects
    label_field : str, optional
        key for the field used for computing labels
    time_start : int, optional
        first day of the period of interest
    time_end : int, optional
        first day after the end of the period of interst
    T : int, optional
        width of the window for the running average
    tau : int, optional
        delay between observation and prediction
    percent : float, optional
        percentage of the most extreme heatwaves
    threshold : float, optional
        if provided overrides `percent`
    
    Returns:
    --------
    X : np.ndarray
        with shape (years, days, lat, lon, field)
    Y : np.ndarray
        with shape (years, days)



In [6]:
help(ln.roll_X)

Help on function roll_X in module Learn2_new:

roll_X(X, roll_axis='lon', roll_steps=64)
    Rolls `X` along a given axis. useful for example for moving France away from the Greenwich meridian.
    In other words this allows one, for example, to shift the grid so that desired areas are not found at the boundary.
    In principle this function allows us to roll along arbitrary axis, including days or years.
    
    Parameters
    ----------
    X : np.ndarray
        with shape (years, days, lat, lon, field)
    roll_axis : int or str, optional
        'year' (or 'y'), 'day' (or 'd'), 'lat', 'lon', 'field' (or 'f')
    roll_steps : int, optional
        number of gridsteps to roll: a positive value for `roll_steps` means that the elements of the array are moved forward in it,
        e.g. `roll_steps` = 1 means that the old first element is now in the second place
        This means that for every axis a positive value of `roll_steps` yields a shift of the array
        'year', 'day' :

So, now let's say you want to call `ln.prepare_data` with just the temperature field, using data from the short 1000 years dataset and rolling `X` by 16 steps, leavingall other values at their default. One (cumbersome) way to do it is the following

In [7]:
X, Y, yp = ln.prepare_data(load_data_kwargs = {'fields': ['t2m'], 'dataset_years': 1000},
                           prepare_XY_kwargs = {'roll_X_kwargs': {'roll_steps': 16}})

prepare_data:
	load_data:


FileNotFoundError: [Errno 2] No such file or directory: b'/local/gmiloshe/PLASIM/Data_Plasim_inter/CONTROL_lsmask.nc'

If this command didn't work it is likely that you are running it from a machine which doesn't have the data in local space, or as mentioned earlier R740server5, in which case you need to specify the source in kwargs by doing:

In [8]:
X, Y, yp = ln.prepare_data(load_data_kwargs = {'fields': ['t2m'], 'dataset_years': 1000, 'mylocal':  '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/PLASIM/'},
                           prepare_XY_kwargs = {'roll_X_kwargs': {'roll_steps': 16}})

prepare_data:
	load_data:
		load_field:
			Loading field tas
			Loaded time array
			input self.var.shape = (150000, 22, 128)
			output self.var.shape = (1000, 150, 22, 128)
		load_field: completed in 21.3 s
		Set_area_integral:
		Set_area_integral: completed in 0.1 s
	load_data: completed in 21.4 s
	prepare_XY:
		make_XY:
			make_X:
			make_X: completed in 0.3 s
			assign_labels:
			assign_labels: completed in 0.0 s
		make_XY: completed in 0.3 s
		roll_X:
		roll_X: completed in 0.4 s
		Mixing
		balance_folds:
			Balancing folds
			fold 0 done!
			fold 1 done!
			fold 2 done!
			fold 3 done!
			fold 4 done!
			fold 5 done!
			fold 6 done!
			fold 7 done!
			fold 8 done!
			fold 9 done!
			Sums of the balanced 10 folds:
			[385 385 385 385 385 385 385 385 385 385]
			std/avg = 0.0
			max relative deviation = 0.0\%
		balance_folds: completed in 0.0 s
		Mixing completed in 0.2 s
		
		X.shape = (1000, 77, 22, 128, 1), Y.shape = (1000, 77)
		Flattened time: X.shape = (77000, 22, 128, 1), Y.

## Better approach

If intstead you already roughly know how the code works, you can proceed in a more elegant way

First create a dictionary of the default parameters using `ln.get_default_params`. Remeber to specify `recursive = True`, which will gather all the default parameters of the functions called in a nested manner

In [9]:
prepare_data_kwargs_default = ln.get_default_params(ln.prepare_data, recursive=True)
print(ut.dict2str(prepare_data_kwargs_default)) # a nice way of printing nested dictionaries

{
    "load_data_kwargs": {
        "dataset_years": 8000,
        "year_list": null,
        "sampling": "",
        "Model": "Plasim",
        "area": "France",
        "filter_area": "France",
        "lon_start": 0,
        "lon_end": 128,
        "lat_start": 0,
        "lat_end": 22,
        "mylocal": "/local/gmiloshe/PLASIM/",
        "fields": [
            "t2m",
            "zg500",
            "mrso_filtered"
        ]
    },
    "prepare_XY_kwargs": {
        "do_premix": false,
        "premix_seed": 0,
        "do_balance_folds": true,
        "nfolds": 10,
        "year_permutation": null,
        "flatten_time_axis": true,
        "make_XY_kwargs": {
            "label_field": "t2m",
            "time_start": 30,
            "time_end": 120,
            "T": 14,
            "tau": 0,
            "percent": 5,
            "threshold": null
        },
        "roll_X_kwargs": {
            "roll_axis": "lon",
            "roll_steps": 64
        }
    }
}


(Again pay attention that the machine we are using has the data in space called 'local'. If so you don't need to specify the variable `mylocal` below)

Now you want to set the two parameters to non default values, and you can do it by using `ut.set_values_recursive`, without needing to account for the level of nestedness of the parameter.

In [12]:
prepare_data_kwargs = ut.set_values_recursive(prepare_data_kwargs_default,
                                              {'fields': ['t2m'], 'dataset_years': 1000, 'roll_steps': 16, 'mylocal': '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/PLASIM/'})
print(ut.dict2str(prepare_data_kwargs))

{
    "load_data_kwargs": {
        "dataset_years": 1000,
        "year_list": null,
        "sampling": "",
        "Model": "Plasim",
        "area": "France",
        "filter_area": "France",
        "lon_start": 0,
        "lon_end": 128,
        "lat_start": 0,
        "lat_end": 22,
        "mylocal": "/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/PLASIM/",
        "fields": [
            "t2m"
        ]
    },
    "prepare_XY_kwargs": {
        "do_premix": false,
        "premix_seed": 0,
        "do_balance_folds": true,
        "nfolds": 10,
        "year_permutation": null,
        "flatten_time_axis": true,
        "make_XY_kwargs": {
            "label_field": "t2m",
            "time_start": 30,
            "time_end": 120,
            "T": 14,
            "tau": 0,
            "percent": 5,
            "threshold": null
        },
        "roll_X_kwargs": {
            "roll_axis": "lon",
            "roll_steps": 16
        }
    }
}


And then you can run

In [13]:
X, Y, yp = ln.prepare_data(**prepare_data_kwargs)

prepare_data:
	load_data:
		load_field:
			Loading field tas
			Loaded time array
			input self.var.shape = (150000, 22, 128)
			output self.var.shape = (1000, 150, 22, 128)
		load_field: completed in 5.6 s
		Set_area_integral:
		Set_area_integral: completed in 0.0 s
	load_data: completed in 5.6 s
	prepare_XY:
		make_XY:
			make_X:
			make_X: completed in 0.2 s
			assign_labels:
			assign_labels: completed in 0.0 s
		make_XY: completed in 0.3 s
		roll_X:
		roll_X: completed in 0.3 s
		Mixing
		balance_folds:
			Balancing folds
			fold 0 done!
			fold 1 done!
			fold 2 done!
			fold 3 done!
			fold 4 done!
			fold 5 done!
			fold 6 done!
			fold 7 done!
			fold 8 done!
			fold 9 done!
			Sums of the balanced 10 folds:
			[385 385 385 385 385 385 385 385 385 385]
			std/avg = 0.0
			max relative deviation = 0.0\%
		balance_folds: completed in 0.0 s
		Mixing completed in 0.2 s
		
		X.shape = (1000, 77, 22, 128, 1), Y.shape = (1000, 77)
		Flattened time: X.shape = (77000, 22, 128, 1), Y.sh

If you want to load your default values from a config file, you can do as following

In [14]:
config_dict = ut.json2dict('example_config.json') # load the config file
print(ut.dict2str(config_dict))
print('\n\n')
config_dict_flat = ut.collapse_dict(config_dict) # flatten the dictionary
print(ut.dict2str(config_dict_flat))

{
    "run_kwargs": {
        "log_level": 20,
        "load_data_kwargs": {
            "dataset_years": 8000,
            "year_list": null,
            "sampling": "",
            "Model": "Plasim",
            "area": "France",
            "filter_area": "France",
            "lon_start": 0,
            "lon_end": 128,
            "lat_start": 0,
            "lat_end": 22,
            "mylocal": "/local/gmiloshe/PLASIM/",
            "fields": [
                "t2m",
                "zg500",
                "mrso_filtered"
            ]
        },
        "prepare_XY_kwargs": {
            "do_premix": false,
            "premix_seed": 0,
            "do_balance_folds": true,
            "nfolds": 10,
            "year_permutation": null,
            "flatten_time_axis": true,
            "make_XY_kwargs": {
                "label_field": "t2m",
                "time_start": 30,
                "time_end": 120,
                "T": 14,
                "tau": 0,
                "pe

In [15]:
ut.set_values_recursive(prepare_data_kwargs_default, config_dict_flat, inplace=True)
nice_dict_print(prepare_data_kwargs_default)

NameError: name 'nice_dict_print' is not defined

We had to do this way because `prepare_data_kwargs` is not a key of the config dictionary. Let's say instead you wanted the default arguments for the function `make_XY`, you could have simply done

**GM**: Where does the function `nice_dict_print` come from?

In [ ]:
make_XY_kwargs_default = ut.extract_nested(config_dict, 'make_XY_kwargs')
print(ut.dict2str(make_XY_kwargs_default))

Where the function `ut.extract_nested` allows to obtain a key from a nested dictionary regardless of the level of indentation

(77000, 22, 128, 1)